In [3]:
# %%
import pandas as pd
import random
from sentence_transformers import SentenceTransformer
import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma

# %%


In [4]:
# ----------------------------
# Step 1: Generate synthetic dataset
# ----------------------------
n_rows = 200
data = {
    "InvoiceNo": [f"INV{1000+i}" for i in range(n_rows)],
    "StockCode": [f"STK{random.randint(100, 999)}" for _ in range(n_rows)],
    "Description": [random.choice(["T-shirt", "Shoes", "Laptop", "Phone", "Book", "Pen", "Bag"]) for _ in range(n_rows)],
    "Quantity": [random.randint(1, 10) for _ in range(n_rows)],
    "InvoiceDate": pd.date_range(start="2023-01-01", periods=n_rows, freq="D").strftime("%Y-%m-%d").tolist(),
    "UnitPrice": [round(random.uniform(5, 500), 2) for _ in range(n_rows)],
    "CustomerID": [random.randint(10000, 20000) for _ in range(n_rows)],
    "Country": [random.choice(["India", "USA", "UK", "Germany", "Canada"]) for _ in range(n_rows)]
}

df = pd.DataFrame(data)
df.to_csv("synthetic_retail_data.csv", index=False)
print("✅ synthetic_retail_data.csv generated successfully!")

✅ synthetic_retail_data.csv generated successfully!


In [11]:
# ----------------------------
# Step 2: Create Parent-Child Chunking (FIXED)
# ----------------------------

# Define product categories
product_categories = {
    "Electronics": ["Laptop", "Phone"],
    "Clothing": ["T-shirt", "Shoes", "Bag"],
    "Stationery": ["Book", "Pen"]
}

# Create parent chunks (category summaries) - FIXED METADATA
parent_chunks = []
for category, products in product_categories.items():
    category_data = df[df["Description"].isin(products)]
    if not category_data.empty:
        avg_price = category_data["UnitPrice"].mean()
        total_quantity = category_data["Quantity"].sum()
        total_transactions = len(category_data)
        
        parent_text = f"Category: {category}. " \
                     f"Products: {', '.join(products)}. " \
                     f"Average Price: ${avg_price:.2f}. " \
                     f"Total Quantity Sold: {total_quantity}. " \
                     f"Total Transactions: {total_transactions}."
        
        parent_chunks.append({
            "text": parent_text,
            "metadata": {
                "chunk_type": "parent",
                "category": category,
                "products": ", ".join(products),  # Convert list to string
                "avg_price": float(avg_price),    # Ensure float type
                "total_quantity": int(total_quantity),  # Ensure int type
                "total_transactions": int(total_transactions)  # Ensure int type
            }
        })

# Create child chunks (individual transactions) - FIXED METADATA
child_chunks = []
for idx, row in df.iterrows():
    chunk_text = f"InvoiceNo: {row['InvoiceNo']}, " \
                 f"StockCode: {row['StockCode']}, " \
                 f"Description: {row['Description']}, " \
                 f"Quantity: {row['Quantity']}, " \
                 f"InvoiceDate: {row['InvoiceDate']}, " \
                 f"UnitPrice: ${row['UnitPrice']}, " \
                 f"CustomerID: {row['CustomerID']}, " \
                 f"Country: {row['Country']}"
    
    # Find which category this product belongs to
    category = None
    for cat, products in product_categories.items():
        if row["Description"] in products:
            category = cat
            break
    
    child_chunks.append({
        "text": chunk_text,
        "metadata": {
            "chunk_type": "child",
            "category": category if category else "Unknown",  # Ensure not None
            "InvoiceNo": str(row["InvoiceNo"]),  # Ensure string
            "StockCode": str(row["StockCode"]),  # Ensure string
            "Description": str(row["Description"]),  # Ensure string
            "Quantity": int(row["Quantity"]),  # Ensure int
            "InvoiceDate": str(row["InvoiceDate"]),  # Ensure string
            "UnitPrice": float(row["UnitPrice"]),  # Ensure float
            "CustomerID": int(row["CustomerID"]),  # Ensure int
            "Country": str(row["Country"])  # Ensure string
        }
    })

In [12]:
# ----------------------------
# Step 3: Create embeddings and store in ChromaDB
# ----------------------------
model = SentenceTransformer("all-MiniLM-L6-v2")

# Combine all chunks
all_chunks = parent_chunks + child_chunks
chunk_texts = [chunk["text"] for chunk in all_chunks]
chunk_metadatas = [chunk["metadata"] for chunk in all_chunks]

embeddings = model.encode(chunk_texts)
print(f"Embeddings shape: {len(embeddings)} x {len(embeddings[0])}")

Embeddings shape: 203 x 384


In [13]:
# ----------------------------
# Step 4: Store in ChromaDB
# ----------------------------
client = chromadb.PersistentClient(path="chromadb_store")
collection = client.get_or_create_collection("retail_parent_child_chunks")

# Clear old data
if collection.count() > 0:
    all_ids = collection.get()["ids"]
    if all_ids:
        collection.delete(ids=all_ids)

# Insert all chunks
for i, (text, emb, metadata) in enumerate(zip(chunk_texts, embeddings, chunk_metadatas)):
    collection.add(
        ids=[str(i)],
        documents=[text],
        embeddings=[emb.tolist()],
        metadatas=[metadata]
    )

print("✅ Parent-child data stored in ChromaDB successfully!")
print("Total stored embeddings:", collection.count())

✅ Parent-child data stored in ChromaDB successfully!
Total stored embeddings: 203


In [14]:
# ----------------------------
# Step 5: Enhanced Query with Parent-Child Context
# ----------------------------
def query_with_context(collection, model, query, k=5):
    query_emb = model.encode([query]).tolist()
    
    # First, search for relevant chunks
    results = collection.query(
        query_embeddings=query_emb,
        n_results=k
    )
    
    print(f"🔎 Query: '{query}'")
    print("=" * 50)
    
    for i, (doc, metadata, distance) in enumerate(zip(results["documents"][0], results["metadatas"][0], results["distances"][0])):
        chunk_type = metadata.get("chunk_type", "unknown")
        print(f"Result {i+1} ({chunk_type}, distance: {distance:.4f}):")
        print(f"   {doc}")
        
        # If it's a child chunk, show which category it belongs to
        if chunk_type == "child":
            category = metadata.get("category", "Unknown")
            print(f"   → Category: {category}")
        
        print("-" * 40)

# %%
# Test the enhanced query
query_with_context(collection, model, "electronics products", k=5)
print("\n")
query_with_context(collection, model, "expensive laptop purchase", k=5)
print("\n")
query_with_context(collection, model, "clothing items", k=5)

🔎 Query: 'electronics products'
Result 1 (parent, distance: 1.2717):
   Category: Electronics. Products: Laptop, Phone. Average Price: $261.66. Total Quantity Sold: 345. Total Transactions: 58.
----------------------------------------
Result 2 (child, distance: 1.4328):
   InvoiceNo: INV1130, StockCode: STK714, Description: Shoes, Quantity: 2, InvoiceDate: 2023-05-11, UnitPrice: $256.48, CustomerID: 17730, Country: India
   → Category: Clothing
----------------------------------------
Result 3 (child, distance: 1.4359):
   InvoiceNo: INV1019, StockCode: STK382, Description: Laptop, Quantity: 3, InvoiceDate: 2023-01-20, UnitPrice: $279.01, CustomerID: 15940, Country: India
   → Category: Electronics
----------------------------------------
Result 4 (child, distance: 1.4424):
   InvoiceNo: INV1133, StockCode: STK212, Description: Laptop, Quantity: 9, InvoiceDate: 2023-05-14, UnitPrice: $442.37, CustomerID: 10421, Country: India
   → Category: Electronics
---------------------------------

In [18]:
# ----------------------------
# Step 6: Advanced Query - Find children of a parent (CORRECTED)
# ----------------------------
def query_category_children(collection, category, limit=10):
    """Find all child transactions for a specific category using direct metadata filtering"""
    # Use get() method with proper $and syntax for multiple conditions
    results = collection.get(
        where={
            "$and": [
                {"chunk_type": "child"},
                {"category": category}
            ]
        },
        limit=limit,
        include=["documents", "metadatas"]  # Include both documents and metadata
    )
    
    print(f"👕 Transactions for category: {category}")
    print("=" * 50)
    
    if not results["documents"]:
        print(f"No transactions found for category: {category}")
        return
    
    total_value = 0
    for i, (doc, metadata) in enumerate(zip(results["documents"], results["metadatas"])):
        price = metadata.get("UnitPrice", 0)
        quantity = metadata.get("Quantity", 0)
        transaction_value = price * quantity
        total_value += transaction_value
        
        print(f"Transaction {i+1}:")
        print(f"   {doc}")
        print(f"   Value: ${transaction_value:.2f}")
        print("-" * 30)
    
    print(f"Total category value: ${total_value:.2f}")
    print(f"Total transactions found: {len(results['documents'])}")

# Test category-specific query
query_category_children(collection, "Electronics")
print("\n")
query_category_children(collection, "Clothing")

👕 Transactions for category: Electronics
Transaction 1:
   InvoiceNo: INV1000, StockCode: STK489, Description: Laptop, Quantity: 7, InvoiceDate: 2023-01-01, UnitPrice: $233.01, CustomerID: 12181, Country: UK
   Value: $1631.07
------------------------------
Transaction 2:
   InvoiceNo: INV1005, StockCode: STK169, Description: Laptop, Quantity: 6, InvoiceDate: 2023-01-06, UnitPrice: $63.98, CustomerID: 17732, Country: Canada
   Value: $383.88
------------------------------
Transaction 3:
   InvoiceNo: INV1009, StockCode: STK709, Description: Laptop, Quantity: 8, InvoiceDate: 2023-01-10, UnitPrice: $182.54, CustomerID: 13084, Country: USA
   Value: $1460.32
------------------------------
Transaction 4:
   InvoiceNo: INV1011, StockCode: STK814, Description: Phone, Quantity: 10, InvoiceDate: 2023-01-12, UnitPrice: $218.05, CustomerID: 12045, Country: Canada
   Value: $2180.50
------------------------------
Transaction 5:
   InvoiceNo: INV1017, StockCode: STK556, Description: Phone, Quantit